In [2]:
import pandas as pd
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime

In [8]:
# 엑셀을 pickle파일로
wd = r'D:\0.전산\2.이관\써머리\data_230204'
for root, dirs, files in os.walk(wd) :
    for f in files :
        df = pd.read_excel(join(wd, f)).fillna("")
        df.to_pickle(join(wd, os.path.splitext(f)[0]+".pkl"))

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
# 파일 명칭 자동 저장
account_f, debt_f, grt_f, law_f, collection_f = "", "", "", "", ""
for root, dirs, files in os.walk(wd) :
    for f in files :
        if re.search(".pkl", f) :
            if re.search("계좌",f) : account_f = f
            elif re.search("채무자",f) : debt_f = f
            elif re.search("보증인",f) : grt_f = f
            elif re.search("법조치",f) : law_f = f
            elif re.search("입금",f) : collection_f = f

In [29]:
account = pd.read_pickle(join(wd, account_f))
debt = pd.read_pickle(join(wd, debt_f))
collection = pd.read_pickle(join(wd,collection_f))
############################
매각사 = "전북은행"
account = account[account.매각사.str.contains(매각사)]
debt = debt[debt.매각사.str.contains(매각사)]
############################
print(len(account), len(debt))

2447 2447


In [31]:
account.시효완성일 = pd.to_datetime(account.시효완성일)
collection.입금일 = pd.to_datetime(collection.입금일)

In [43]:
raw_data = pd.DataFrame(None)
raw_data["계좌키"] = account.계좌키.apply(str)
raw_data["채무자키"] = account.채무자키.apply(str)
raw_data["타채무자키"] = account.타채무자키.apply(str)
raw_data["채권구분"] = account.채권구분
raw_data["매각사"] = account.매각사
raw_data["채무상태"] = account.채무상태
raw_data["담당자"] = account.담당자
raw_data["성명"] = account.채무자명
raw_data["주민등록번호"] = account.채무자주민번호.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계
raw_data["시효완성일"] = account.시효완성일

In [44]:
debt.채무자키 = debt.채무자키.apply(str)
debt = debt[["채무자키", "개인법인", "연령", "자택우편번호", "자택주소"]]
raw_data = pd.merge(raw_data, debt, on="채무자키", how="inner")
raw_data.개인법인.isnull().value_counts()

False    2447
Name: 개인법인, dtype: int64

In [45]:
raw_data.to_excel(join(wd, "원데이터_"+datetime.today().strftime("%Y%m%d")+".xlsx"), index=False)

In [42]:
raw_data["주민등록번호"]

0       611017-1
1       611017-1
2       611017-1
3       611017-1
4       411223-2
          ...   
2442    640220-2
2443    821228-1
2444    821228-1
2445    401-81-2
2446    401-81-2
Name: 주민등록번호, Length: 2447, dtype: object